In [2]:
import os 
import sys 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sn 

home = \
    os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
pths = [os.path.join(home, 'astrobot'), 
        os.path.join(home, 'astrobot', 'tests')]

for pth in pths: 
    if pth not in sys.path: sys.path.insert(0, pth)
    
import astrobot as ab 

Create categorical conditional matrices for: 
  like_x_y, prior_y, then:k 

XY: R~3x2 and Y: R~3x1 
Yi = rows 3 observations 
Xj = cols 2 states

X xj, xj, xj   Y
 [[0,  0,  0], yi
  [0,  0,  0]] yi

Matrix multiply XY, Y => 
Ex[Y] = XY @ Y = \sum_k{ XY[:,k=j] * Y[k=i,0] }
 
joint_x_y: p(x,y) = outer(X, Y); iff X, Y are independent
margin_X: p(X) = like_x_y.T @ Y 
margin_Y: p(Y) = sum(joint_x_y, axis=1~{obs_0, obs_1, obs_2})
               = like_x_y @ Y 
post_Y_x: p(Y|x) = (like_x_y @ prior_Y) / margin_X    

In [3]:
"""
Marginal Y from matrix multiply of like XY and prior Y.
     
    $$
    p(y) &= \sum_i{ p(x|y_i) \cdot p(y_i) } \
         &= p(x|y_i) \cdot p(y_i)  
    $$
"""
    
    
# p(y) = p(x|y).T p(y) 
# ~ 3x2 x 2x1 = 2x1; expectation of y    
# Create liklihood matrix 
like_x_y = np.array(
    [[0.6, 0.15, 0.25],
     [0.5, 0.41, 0.09]]
    )  

# Sum over axis to get marginal evidence, states 
# In likelihood, each state == 1, as it is a probability.
# In joint, each state != 1
print('Sum over likelihood evidence axis 0 =>', like_x_y.sum(axis=0))
print('Sum over likelihood state axis 1 =>', like_x_y.sum(axis=1))

# x-axis=obs, y-axis=states
prior_y = np.array(
    [[0.75],
     [0.25]]
    )

tst_margin_X = np.array(
    [[0.575],
     [0.215], 
     [0.21]]
    )

assert like_x_y.shape == (2, 3), like_x_y
assert prior_y.shape == (2, 1), prior_y
margin_X = like_x_y.T @ prior_y # (2,3) x (3,1)
assert margin_X.shape == (3, 1), margin_X.shape
assert np.allclose(margin_X, tst_margin_X, atol=1e-3), margin_X

# Check correct pmf
# Test likelihood is pmf
tst_state_pmf = np.array([1., 1.]) 
state_pmf = like_x_y.sum(axis=1) 
assert np.allclose(state_pmf, tst_state_pmf, atol=1e-3), state_pmf
obs_pmf = like_x_y.sum()
assert np.abs(obs_pmf - 1.0) > 1e-3, obs_pmf
# Test prior is pmf
assert np.abs(prior_y.sum() - 1.0) < 1e-3, prior_y.sum()

print('Success!')

Sum over likelihood evidence axis 0 => [1.1  0.56 0.34]
Sum over likelihood state axis 1 => [1. 1.]
Success!


In [ ]:
from dataclasses import dataclass 
from typing import Sequence, Union 
from numpy.typing import npt.NDArray
import functools as ft 

@dataclass(init=True, repr=True, eq=True, order=False, frozen=True, unsafe_hash=False)
class Pmf:
    """Probability Mass Function.    
    - Make simple Pmf dataclass
    - arg: data=mtx of values, columns[opt]=[dim1, dim2]
    - if opt: make pandas df_joint of all 2 combos
    - .m = data
    - .cols = cols
    - add indexing: [dim1:dim2] => DataFrame(data[dim1, dim2]).joint
    - check sums to 1.0 ~margin/joint
    - check sums to 1.0 on at least on axis (~likelihood)
    - .type: margin, joint, like
    - set to frozen
    """
    data: npt.NDArray[float]
    cols: Sequence[str]
    
    @ft.singledispatchmethod
    
    
     
    